In [101]:
import os
from tqdm import tqdm
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import matplotlib.pyplot as plt

In [102]:
# read nuts shp and enter the path for
nuts = gpd.read_file('/data/xiang/1-Data/NUTS/NUTS_RG_01M_2021_3035.shp')
read_folder = '/data/xiang/1-Data/Urban atlas'

In [103]:
for folder in tqdm(os.listdir(read_folder)):
    # get the country id and ua id of each shp
    country_id = folder.split('_')[0][:2]
    ua_num = folder.split('_')[0][2:5]

    # read the shp and separate green area from the total area
    sub_folder = read_folder + r'/' + folder + r'/' + 'Data' + r'/'
    ua = gpd.read_file(sub_folder + os.listdir(sub_folder)[0])
    ua.to_crs('EPSG:3035', inplace = True)
    green = ua[ua['class_2018']=='Green urban areas']
    
    # get the total area and total green area of shp that falls within the boundary of NTUS regions in the same country
    country_shp = nuts[nuts['NUTS_ID'].str.contains(country_id)]
    dict_temp = {}
    for subregion_id in country_shp['NUTS_ID']:
        subregion = country_shp[country_shp['NUTS_ID']==subregion_id]
        Green_temp = gpd.sjoin(green, subregion, how='inner')
        Green_temp['shape area'] = Green_temp.area
        total_temp = gpd.sjoin(ua, subregion, how='inner')
        total_temp['shape area'] = total_temp.area
        dict_temp[subregion_id] = [Green_temp['shape area'].sum(),total_temp['shape area'].sum()]

    # record the green and total area and export the dataframe
    df_temp = pd.DataFrame([dict_temp]).T
    df_temp['green area'] = df_temp[0].apply(lambda x: x[0])
    df_temp['total area'] = df_temp[0].apply(lambda x: x[1])
    df_temp[['green area','total area']].to_excel('/data/xiang/3-case studies/1-urban atlas/FUAs' + r'/' + country_id + '_' + ua_num + '.xlsx')

100%|███████████████████████████████████████| 788/788 [6:44:49<00:00, 30.82s/it]


In [104]:
file_list = os.listdir('/data/xiang/3-case studies/1-urban atlas/FUAs')
country_set = set([i.split('_')[0] for i in file_list if i.endswith('.xlsx')])
for country in country_set:
    file_country_list = [i for i in file_list if i.split('_')[0] == country]
    df_comb = None
    for file in file_country_list:
        if file.endswith('.xlsx'):
            # for FUAs in the same country, sum the total of green and total areas to each NUTS level
            df_temp = pd.read_excel('/data/xiang/3-case studies/1-urban atlas/FUAs' + r'/' + file, index_col=0)
            df_comb = df_temp if df_comb is None else df_comb + df_temp
        else:
            pass
    df_comb.to_excel('/data/xiang/3-case studies/1-urban atlas/country merge' + r'/' + country + '.xlsx')

In [105]:
df_concat = None
for file in os.listdir('/data/xiang/3-case studies/1-urban atlas/country merge'):
    df_temp = pd.read_excel('/data/xiang/3-case studies/1-urban atlas/country merge' + r'/' + file)
    df_concat = df_temp if df_concat is None else pd.concat([df_concat, df_temp])

/tmp/ipykernel_5640/2005940219.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = df_temp if df_concat is None else pd.concat([df_concat, df_temp])
/tmp/ipykernel_5640/2005940219.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = df_temp if df_concat is None else pd.concat([df_concat, df_temp])


In [106]:
df_concat.columns = ['geo', 'green urban area', 'total functional urban area']
df_concat['fraction green urban area'] = 100*df_concat['green urban area']/df_concat['total functional urban area']
df_concat['obsTime'] = '2018'
df_concat['freq'] = '6 years'

reshap_df = pd.melt(df_concat, id_vars=['geo', 'freq', 'obsTime'], value_vars = ['green urban area', 'total functional urban area', 'fraction green urban area'], var_name = 'indicator', value_name = 'obsValue')
reshap_df['unit'] = reshap_df['indicator'].apply(lambda x: '%' if x == 'fraction green urban area' else 'square meters')
reshap_df.to_csv('/data/xiang/3-case studies/1-urban atlas/Urban green area.csv')

In [107]:
reshap_df

,geo,freq,obsTime,indicator,obsValue,unit
0,DK,6 years,2018,green urban area,1.210867e+08,square meters
1,DK0,6 years,2018,green urban area,1.210867e+08,square meters
2,DK01,6 years,2018,green urban area,5.431200e+07,square meters
3,DK02,6 years,2018,green urban area,1.010140e+07,square meters
4,DK03,6 years,2018,green urban area,1.588386e+07,square meters
...,...,...,...,...,...,...
6061,CZ041,6 years,2018,fraction green urban area,4.271086e-01,%
6062,CZ063,6 years,2018,fraction green urban area,2.527936e-01,%
6063,CZ064,6 years,2018,fraction green urban area,3.434801e-01,%
6064,CZ032,6 years,2018,fraction green urban area,2.117445e-01,%
